In [16]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
import time

In [34]:
URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

response = requests.get(URL)

In [41]:
state = re.search(
    r'[A-Z][a-z]+(?: [A-Z][a-hj-z][a-z]+)?', 
    BS(response.text).findAll('title')[0].text 
)[0]
type(state)

str

In [36]:
district = re.search(
    r'District [0-9]{2}', 
    BS(response.text).findAll('title')[0].text
)[0]
type(district)

str

In [37]:
state_dict = {
    'State' : [state],
    'District' : [district]
}

In [38]:
pd.DataFrame.from_dict(state_dict)

,State,District
0,Tennessee,District 07


In [3]:
def district_scraper(state, district):
    URL = f'https://www.opensecrets.org/races/candidates?cycle=2020&id={state}{district}&spec=N'
    soup = BS(requests.get(URL).text)
    
    candidate_list = [x.text.strip() for x in soup.find_all('strong')]
    finances = (pd
        .read_html(
            str(soup.findAll('table', attrs={'class' : 'Members--table'}))
                .replace('$','')
                .replace(':', '')
        )
    )
    
    
    temp_df = pd.DataFrame()
    
    temp_df['name'] = [re.findall(r'(.+ .+) \(\w', item)[0] for item in candidate_list]
    
    temp_df['party'] = [re.findall(r'\((\w)\)', item)[0] for item in candidate_list]
    
    temp_df['state'] = str(re.search(r'[A-Z][a-z]+(?: [A-Z][a-hj-z][a-z]+)?',
                                     soup.findAll('title')[0].text)[0])
    
    temp_df['district'] = str(re.findall(r'District ([0-9]{2})', soup.findAll('title')[0].text)[0])
    
    temp_df['incumbent'] = ['Incumbent' if re.search(r'(Incumbent)', str(item)) != None 
                            else 'Not Incumbent' for item in candidate_list]
    
    temp_df['winner'] = ['Winner' if re.search(r'(Winner)', str(item)) is not None
                              else 'Not Winner' for item in candidate_list]
    
    temp_df['vote'] = [re.findall(r'(\d*[.?]\d*\%)', item)[0] for item in candidate_list]
    
    temp_df['raised'] = [finances[i].iloc[0,1] for i in range(len(candidate_list))]
    
    temp_df['spent'] = [finances[i].iloc[1,1] for i in range(len(candidate_list))]
    
    temp_df['cash_on_hand'] = [finances[i].iloc[2,1] for i in range(len(candidate_list))]
    
    global candidate_df
    candidate_df = pd.concat([candidate_df, temp_df])
    candidate_df = candidate_df.reset_index(drop=True)

In [4]:
def state_candidate(state, district = all):
    global candidate_df
    candidate_df = pd.DataFrame()
    district_dict = {'TN' : 9, 'ND' : 1, 'MI' : 14, 'OK' : 5}
    abbre_list = list(district_dict.keys())
    if district == all:
        for num in range(1, district_dict[state] + 1):
            district_scraper(str(state), str(num).zfill(2))
    else:
        district_scraper(str(state), str(district).zfill(2))
    return candidate_df

In [5]:
state_candidate('OK', all)

,name,party,state,district,incumbent,winner,vote,raised,spent,cash_on_hand
0,Kevin Hern,R,Oklahoma,01,Incumbent,Winner,63.7%,1094104,830366,345198
1,Kojo Asamoa-Caesar,D,Oklahoma,01,Not Incumbent,Not Winner,32.7%,157355,173928,-5637
2,Markwayne Mullin,R,Oklahoma,02,Incumbent,Winner,75.0%,1660206,1077939,610310
3,Danyell Lanier,D,Oklahoma,02,Not Incumbent,Not Winner,22.0%,41730,41722,7
4,Frank D Lucas,R,Oklahoma,03,Incumbent,Winner,78.5%,872749,1036145,282179
5,Tom Cole,R,Oklahoma,04,Incumbent,Winner,67.8%,1812530,1753697,1277189
6,Mary Brannon,D,Oklahoma,04,Not Incumbent,Not Winner,28.8%,-2084,1599,1734
7,Kendra Horn,D,Oklahoma,05,Incumbent,Not Winner,47.9%,5949800,5946195,103097
8,Stephanie Bice,R,Oklahoma,05,Not Incumbent,Winner,52.1%,3720520,3508929,211591


In [25]:
district_dict = {'AL': 7,
 'AK': 1,
 'AZ': 9,
 'AR': 4,
 'CA': 53,
 'CO': 7,
 'CT': 5,
 'DE': 1,
 'FL': 27,
 'GA': 14,
 'HI': 2,
 'ID': 2,
 'IL': 18,
 'IN': 9,
 'IA': 4,
 'KS': 4,
 'KY': 6,
 'LA': 6,
 'ME': 2,
 'MD': 8,
 'MA': 9,
 'MI': 14,
 'MN': 8,
 'MS': 4,
 'MO': 8,
 'MT': 1,
 'NE': 3,
 'NV': 4,
 'NH': 2,
 'NJ': 12,
 'NM': 3,
 'NY': 27,
 'NC': 13,
 'ND': 1,
 'OH': 16,
 'OK': 5,
 'OR': 5,
 'PA': 18,
 'RI': 2,
 'SC': 7,
 'SD': 1,
 'TN': 9,
 'TX': 36,
 'UT': 4,
 'VT': 1,
 'VA': 11,
 'WA': 10,
 'WV': 3,
 'WI': 8,
 'WY': 1}

In [33]:
def state_candidate(state = all, district = all):
    global candidate_df
    candidate_df = pd.DataFrame()
    global district_dict

    if state == all and district == all:
        for s, d in district_dict.items():
            time.sleep(1.5)
            for num in range(1, d + 1):
                district_scraper(str(s), str(num).zfill(2))
    
    elif district == all:
        for num in range(1, district_dict[state] + 1):
            district_scraper(str(state), str(num).zfill(2))
    
    elif state == all:
        for s, d in district_dict.items():
            try:   
                district_scraper(str(s), str(district).zfill(2))
            except:
                pass
    
    else:
        district_scraper(str(state), str(district).zfill(2))
    
    return candidate_df

In [35]:
state_candidate(all, 10)

,name,party,state,district,incumbent,winner,vote,raised,spent,cash_on_hand
0,Josh Harder,D,California,10,Incumbent,Winner,55.2%,7400135,3950613,3469763
1,Ted Howze,R,California,10,Not Incumbent,Not Winner,44.8%,1426552,1721882,14241
2,Val Demings,D,Florida,10,Incumbent,Winner,63.6%,1876047,1191020,771422
3,Vennia Francois,R,Florida,10,Not Incumbent,Not Winner,36.4%,335480,332852,9289
4,Kristofer Steven Lawson,I,Florida,10,Not Incumbent,Not Winner,0.0%,47,47,0
5,Jody Hice,R,Georgia,10,Incumbent,Winner,62.3%,595947,457648,249916
6,Tabitha Johnson-Green,D,Georgia,10,Not Incumbent,Not Winner,37.7%,14808,12214,-8
7,Brad Schneider,D,Illinois,10,Incumbent,Winner,63.8%,3459512,2873333,1873763
8,Valerie Ramirez Mukherjee,R,Illinois,10,Not Incumbent,Not Winner,36.2%,130225,64764,65461
9,Bill Keating,D,Massachusetts,10,Not Incumbent,Winner,46.9%,1508869,1505516,3353
